In [43]:
import pandas as pd

def remove_zero_depth_entries(labels_file_path, output_file):
    # Load the file into a DataFrame
    column_names = [
        "frame", "track_id", "type", "bbox_left", "bbox_top",
        "bbox_right", "bbox_bottom", "height", "width", "length",
        "location_x", "location_y", "location_z", "rotation_y"
    ]
    data = pd.read_csv(labels_file_path, sep=" ", header=None, names=column_names)

    # Remove rows where location_z is 0
    data = data[data['location_z'] != 0]

    # Save the filtered file
    data.to_csv(output_file, sep=" ", header=False, index=False)
    print(f"Filtered file saved to: {output_file}")

In [4]:
# Function to plot the trajectories in 3D
def plot_trajectories_in_3d_interactive(df, num_track_ids=None):
    """
    Plots interactive 3D trajectories of specified track IDs using the 'location' data from the labels file.
    Y is treated as the vertical axis.

    Parameters:
    df (DataFrame): DataFrame containing the labels data with columns ['frame', 'track id', 'location_x', 'location_y', 'location_z'].
    num_track_ids (int, optional): Number of random track IDs to plot. If None, plot all track IDs.

    Returns:
    None
    """
    # Get unique track IDs
    unique_track_ids = df['track id'].unique()

    # Select a random subset if num_track_ids is specified
    if num_track_ids is not None:
        if num_track_ids > len(unique_track_ids):
            raise ValueError(f"Requested {num_track_ids} track IDs, but only {len(unique_track_ids)} are available.")
        selected_track_ids = random.sample(list(unique_track_ids), num_track_ids)
    else:
        selected_track_ids = unique_track_ids

    # Filter the DataFrame to include only the selected track IDs
    df = df[df['track id'].isin(selected_track_ids)]

    # Define a color palette
    color_map = px.colors.qualitative.Light24
    color_dict = {track_id: color_map[i % len(color_map)] for i, track_id in enumerate(selected_track_ids)}

    # Prepare the data for Plotly
    data_traces = []
    for track_id in selected_track_ids:
        track_df = df[df['track id'] == track_id].sort_values('frame')
        x_coords = track_df['location_x']
        y_coords = track_df['location_y']
        z_coords = track_df['location_z']
        frames = track_df['frame']

        color = color_dict[track_id]

        # Create the line trace for the trajectory
        trace_line = go.Scatter3d(
            x=x_coords,
            y=z_coords,  # Z is plotted on the horizontal plane
            z=y_coords,  # Y is the vertical axis
            mode='lines',
            name=f'Track ID {track_id}',
            line=dict(color=color, width=4),
            hoverinfo='text',
            text=[f'Track ID: {track_id}<br>Frame: {frame}<br>X: {x:.2f}<br>Y: {y:.2f}<br>Z: {z:.2f}' 
                  for frame, x, y, z in zip(frames, x_coords, y_coords, z_coords)]
        )

        # Mark the starting point
        trace_start = go.Scatter3d(
            x=[x_coords.iloc[0]],
            y=[z_coords.iloc[0]],
            z=[y_coords.iloc[0]],
            mode='markers+text',
            marker=dict(color='green', size=8, symbol='circle'),
            name=f'Start of {track_id}',
            hoverinfo='text',
            text=[f'Start<br>Track ID: {track_id}<br>Frame: {frames.iloc[0]}'],
            textposition='top center',
            showlegend=False
        )

        # Mark the ending point
        trace_end = go.Scatter3d(
            x=[x_coords.iloc[-1]],
            y=[z_coords.iloc[-1]],
            z=[y_coords.iloc[-1]],
            mode='markers+text',
            marker=dict(color='red', size=8, symbol='x'),
            name=f'End of {track_id}',
            hoverinfo='text',
            text=[f'End<br>Track ID: {track_id}<br>Frame: {frames.iloc[-1]}'],
            textposition='top center',
            showlegend=False
        )

        # Add the traces to the data list
        data_traces.extend([trace_line, trace_start, trace_end])

    # Define the layout of the plot
    layout = go.Layout(
        title='Interactive 3D Trajectories of Track IDs',
        scene=dict(
            xaxis=dict(title='X (meters)'),
            yaxis=dict(title='Z (meters)'),  # Z is horizontal plane axis
            zaxis=dict(title='Y (meters)'),  # Y is vertical axis
            aspectmode='data'  # Ensures equal scaling for all axes
        ),
        legend=dict(
            x=0.8,
            y=0.9,
            bgcolor='rgba(255,255,255,0.5)',
            bordercolor='black'
        ),
        margin=dict(l=0, r=0, b=0, t=30)
    )

    # Create the figure and display it
    fig = go.Figure(data=data_traces, layout=layout)
    fig.update_layout(scene_camera=dict(eye=dict(x=1.2, y=1.2, z=0.8)))
    fig.show()

In [44]:
import pandas as pd
import numpy as np

# Example usage:
labels_file_path = r'C:\Users\User\Documents\GitHub\pfas_finalproject\stereo_preds_seq_02_all.txt'
output_file = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_02.txt'
remove_zero_depth_entries(labels_file_path, output_file)

Filtered file saved to: C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_02.txt


In [52]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import random

# Define the file path
labels_file_path = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_02.txt' 
# Read the labels file
headers = [
    "frame", "track id", "type",
    "bbox_left", "bbox_top", "bbox_right", "bbox_bottom",
    "height_3d", "width_3d", "length_3d",
    "location_x", "location_y", "location_z",
    "rotation_y"
]
df = pd.read_csv(labels_file_path, sep=' ', names=headers)

In [51]:
# Call the plotting function
plot_trajectories_in_3d_interactive(df)